# MiniProyecto 4: Optimización de parámetros

¡Bienvenidos al cuarto (mini)proyecto de la carrera de Data Science de Acamica! 

En este proyecto vamos a seguir trabajando (por última vez) con el dataset de propiedades en venta publicadas en el portal [Properati](www.properati.com.ar). El objetivo en este caso es optimizar los parámetros de los algoritmos que usamos en el proyecto pasado.

El dataset es el mismo del proyecto 3. Recordemos que las columnas que se agregan son:

* `barrios_match`: si coincide el barrio publicado con el geográfico vale 1, si no 0.

* `PH`, `apartment`, `house`: variables binarias que indican el tipo de propiedad.

* dummies de barrios: variables binarias con 1 o 0 según el barrio.

La métrica que vamos a usar para medir es RMSE (raíz del error cuadréatico medio), cuya fórmula es:

$$RMSE = \sqrt{\frac{\sum_{t=1}^n (\hat y_t - y_t)^2}{n}}$$

## Pandas - Levantamos el dataset

In [1]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
path_dataset = 'dataset/datos_properati_limpios_model.csv'
df = pd.read_csv(path_dataset)

**Separá** el dataset en entrenamiento (80%) y test (20%) utilizando como target la columna `price_aprox_usd`

In [ ]:
# Hacé la separación en esta celda

In [2]:
# Esto es lo que hace
import numpy as np
np.random.seed(123)
from sklearn.model_selection import train_test_split
X = df.drop(['price_aprox_usd'], axis=1)
y = df['price_aprox_usd']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape[0], X_test.shape[0])

5100 1276


## Scikit-learn - Entrenamiento

Para repasar los parámetros de árboles de decisión en Scikit-learn: 

http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

En primer lugar veamos como hacer cross validation. Para eso necesitamos definir la cantidad de folds, en este caso vamos a usar 5.

GridSearchCV nos permite testear a través de un espacio de búsqueda de parámetros la mejor combinación posible dado un estimador.

Por ejemplo, en este caso probamos la profundidad máxima y la máxima cantidad de features para hacer los split. Ambos entre 1 y 5.
Recordemos que para hacer la optimización scikit-learn usa la métrica `neg_mean_squared_error` en lugar de `mean_squared_error`.

**Creá** una variable `param_grid` con valores del 1 al 5 para los atributos `max_depth` y `max_features`. 

In [3]:
# Creá en esta celda la variable param_grid
param_grid = {'max_depth': [1, 2, 3, 4, 5], 'max_features': [1, 2, 3, 4, 5]}

**Importá** `GridSearchCV` y `DecisionTreeRegressor`.

**Creá** una variable `grid_search` y asignale un `GridSearchCV` que recorra el `param_grid` que creaste con el algoritmos `DecisionTreeRegressor` y el un scoring de `neg_mean_squared_error`

In [4]:
# Importa y crea un GridSearchCV en esta celda
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

A continuación, realizá el `fit` del `grid_search` con el conjunto de entrenamiento

In [5]:
# Hace el fit de grid search en esta celda
modelo = DecisionTreeRegressor()
grid_search = GridSearchCV(modelo, param_grid, cv=5,
                           scoring='neg_mean_squared_error', 
                           return_train_score=True)
grid_search.fit(X_train, y_train)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'max_depth': [1, 2, 3, 4, 5], 'max_features': [1, 2, 3, 4, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

Revisemos los resultados. Recordemos que no están expresados en RMSE.

In [12]:
grid_search.scorer_(grid_search, X_train, y_train)

-947511458.6600356

**Mostrá** los `grid_scores` obtenidos durante el `grid_search`

In [7]:
# Mostrá los grid_scores en esta celda
grid_search.cv_results_

{'mean_fit_time': array([0.00779519, 0.00579829, 0.0026093 , 0.00625024, 0.00673261,
        0.00959444, 0.00412469, 0.00937471, 0.00625052, 0.00312519,
        0.00813661, 0.00412545, 0.00625181, 0.00625081, 0.003125  ,
        0.00625062, 0.00312481, 0.00625014, 0.00786533, 0.00619607,
        0.00599704, 0.00639758, 0.0065959 , 0.00659575, 0.00612369]),
 'std_fit_time': array([1.59923453e-03, 3.98375013e-04, 2.49477980e-03, 7.65494745e-03,
        8.28091641e-03, 5.27172525e-03, 6.06837167e-03, 7.65442186e-03,
        7.65529785e-03, 6.25038147e-03, 7.90451662e-03, 6.06923309e-03,
        7.65687466e-03, 7.65564866e-03, 6.25000000e-03, 7.65541468e-03,
        6.24961853e-03, 7.65483068e-03, 7.12488505e-03, 3.99209179e-04,
        7.47889859e-07, 4.89804070e-04, 4.89434458e-04, 7.99369855e-04,
        5.82362842e-03]),
 'mean_score_time': array([0.0016006 , 0.00139947, 0.00372524, 0.00312481, 0.0002007 ,
        0.00199976, 0.00353088, 0.0031251 , 0.        , 0.        ,
        0.00

De esta manera, el valor con mejor resultado (dado el espacio de búsqueda definido) es `max_depth` 3 y `max_features` 3.

**Mostrá** el mejor score y los mejores parámetros encontrados por `grid_search`

In [8]:
# Mostrás los resultados en esta celda
grid_search.best_params_

{'max_depth': 4, 'max_features': 4}

Convertimos a RMSE.

In [14]:
score = grid_search.best_score_
score

-713698124.9712428

In [15]:
def nmsq2rmse(score):
    return np.round(np.sqrt(-score), 2)
nmsq2rmse(score)

26715.13

__Encontrar el mejor modelo para el espacio de búsqueda dado__

* `"min_samples_split": [2, 10, 20]`
* `"max_depth": [None, 2, 5, 10, 15]`
* `"min_samples_leaf": [1, 5, 10, 15]`
* `"max_leaf_nodes": [None, 5, 10, 20]`

Recordemos que `GridSearchCV` tiene como parámetro default `refit=True`. Esto significa que luego de hacer la corrida se ajusta el mejor modelo al conjunto de datos de entrada. De esta manera, se puede predecir directamente usando `best_estimator_`.

In [16]:
new_param_grid = {'max_depth': [None, 2, 5, 10, 15],"min_samples_split": [2, 10, 20], "min_samples_leaf": [1, 5, 10, 15], "max_leaf_nodes": [None, 5, 10, 20]  }

In [17]:
grid_search = GridSearchCV(modelo, new_param_grid, cv=5,
                           scoring='neg_mean_squared_error', 
                           return_train_score=True)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'max_depth': [None, 2, 5, 10, 15], 'min_samples_split': [2, 10, 20], 'min_samples_leaf': [1, 5, 10, 15], 'max_leaf_nodes': [None, 5, 10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

In [19]:
optimised_decision_tree = grid_search.best_estimator_
optimised_decision_tree 

DecisionTreeRegressor(criterion='mse', max_depth=10, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=15,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

__Evaluemos en testing el desempeño de este modelo.__

Como venimos trabajando, el resultado en testing será la medición que usaremos como benchmark para comparar este modelos con otros en el futuro, puesto que no estuvo en contacto con el dataset de test para la calibración.

In [20]:
from sklearn.metrics import mean_squared_error
y_opt_pred = optimised_decision_tree.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_opt_pred))
np.round(rmse)

21301.0

Vemos los primeros 10 resultados de la predicción del valor de propiedades.

In [21]:
val_real = pd.Series(y_test.values)
val_pred = pd.Series(y_opt_pred)

In [22]:
predicciones = pd.concat([val_real.rename('Valor real'),val_pred.rename('Valor Pred') ,abs(val_real-val_pred).rename('Dif(+/-)')] ,  axis=1)

In [23]:
predicciones.head(10)

,Valor real,Valor Pred,Dif(+/-)
0,80000.000,103438.660,23438.660
1,128000.000,135705.882,7705.882
2,150000.000,156075.759,6075.759
3,85000.000,102400.991,17400.991
4,135000.000,135571.622,571.622
5,135000.000,109560.000,25440.000
6,68000.000,75181.250,7181.250
7,110000.000,140444.444,30444.444
8,134000.000,158431.250,24431.250
9,110000.000,76701.202,33298.798
